In [1]:
import numpy as np

In [2]:
import pickle as pk

In [3]:
T = 96
dT = 4
num_cluster = 1600

In [4]:
data = dict({})

for m in range(10, 12):
    for d in range(1, 29):
        filename = '/data/fan/UsersInTokyoProcessed/201210{:02d}_interp.pk'.format(d)
        print(filename)
        with open(filename, 'rb') as f:
            data[(m, d)] = pk.load(f)

/data/fan/UsersInTokyoProcessed/20121001_interp.pk
/data/fan/UsersInTokyoProcessed/20121002_interp.pk
/data/fan/UsersInTokyoProcessed/20121003_interp.pk
/data/fan/UsersInTokyoProcessed/20121004_interp.pk
/data/fan/UsersInTokyoProcessed/20121005_interp.pk
/data/fan/UsersInTokyoProcessed/20121006_interp.pk
/data/fan/UsersInTokyoProcessed/20121007_interp.pk
/data/fan/UsersInTokyoProcessed/20121008_interp.pk
/data/fan/UsersInTokyoProcessed/20121009_interp.pk
/data/fan/UsersInTokyoProcessed/20121010_interp.pk
/data/fan/UsersInTokyoProcessed/20121011_interp.pk
/data/fan/UsersInTokyoProcessed/20121012_interp.pk
/data/fan/UsersInTokyoProcessed/20121013_interp.pk
/data/fan/UsersInTokyoProcessed/20121014_interp.pk
/data/fan/UsersInTokyoProcessed/20121015_interp.pk
/data/fan/UsersInTokyoProcessed/20121016_interp.pk
/data/fan/UsersInTokyoProcessed/20121017_interp.pk
/data/fan/UsersInTokyoProcessed/20121018_interp.pk
/data/fan/UsersInTokyoProcessed/20121019_interp.pk
/data/fan/UsersInTokyoProcessed

In [5]:
global_dist = np.zeros([num_cluster, T], dtype=np.float32)

In [6]:
uid_set = set([])
for d in data:
    uid_set = uid_set | set(list(data[d].keys()))

In [7]:
local_dist = dict({})
for uid in uid_set:
    local_dist[uid] = np.zeros([num_cluster, T], dtype=np.float32)

In [8]:
for d in data:
    print(d)
    for uid in data[d]:
        for t in range(2 * dT, T):
            local_dist[uid][data[d][uid][t], t] += 1

(10, 1)
(10, 2)
(10, 3)
(10, 4)
(10, 5)
(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 10)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(10, 16)
(10, 17)
(10, 18)
(10, 19)
(10, 20)
(10, 21)
(10, 22)
(10, 23)
(10, 24)
(10, 25)
(10, 26)
(10, 27)
(10, 28)
(11, 1)
(11, 2)
(11, 3)
(11, 4)
(11, 5)
(11, 6)
(11, 7)
(11, 8)
(11, 9)
(11, 10)
(11, 11)
(11, 12)
(11, 13)
(11, 14)
(11, 15)
(11, 16)
(11, 17)
(11, 18)
(11, 19)
(11, 20)
(11, 21)
(11, 22)
(11, 23)
(11, 24)
(11, 25)
(11, 26)
(11, 27)
(11, 28)


In [9]:
for uid in local_dist:
    global_dist += local_dist[uid]

In [10]:
global_dist += 0.5

In [11]:
global_dist /= np.sum(global_dist, axis=0)

In [12]:
date_list = [(2012, m, d) for m in range(12, 13) for d in range(1, 32)] + [(2013, m, d) for m in range(1, 3) for d in range(1, 32)]

In [13]:
import os

In [15]:
metrics_dict = dict({})

for y, m, d in date_list:
    filename = '/data/fan/UsersInTokyoProcessed/{:04d}{:02d}{:02d}_interp.pk'.format(y, m, d)
    if not os.path.isfile(filename):
        continue
    print(filename)

    avg_loss = 0.0
    mrr = 0.0
    avg_rank = 0.0
    acc5 = 0.0
    
    with open(filename, 'rb') as f:
        data = pk.load(f)
        
    cnt = 0
    for uid in data:
        if uid in local_dist:
            if np.random.ranf() < 0.2:
                dist = 0.6 * (local_dist[uid] + 1e-4) / (np.sum(local_dist[uid], axis=0) + 1e-4) + 0.4 * global_dist
                for t in range(2 * dT, T):
                    cnt += 1
                    avg_loss += -np.log(dist[data[uid][t], t])
                    rank = np.where((np.argsort(-dist[:, t]).T - data[uid][t]).T == 0)[0] + 1
                    mrr += np.mean(1 / rank)
                    avg_rank += np.mean(rank)
                    acc5 += np.mean(rank <= 5)
                
    
    avg_loss /= cnt
    mrr /= cnt
    avg_rank /= cnt
    acc5 /= cnt
    print(y, m, d, avg_loss, mrr, avg_rank, acc5)
    metrics_dict[(y, m, d)] = (avg_loss, mrr, avg_rank, acc5)

/data/fan/UsersInTokyoProcessed/20121201_interp.pk
2012 12 1 3.3041999275062275 0.5584707496133254 129.328559622196 0.6897225501770956
/data/fan/UsersInTokyoProcessed/20121202_interp.pk
2012 12 2 3.34368893869176 0.546077038407195 128.89247451166057 0.6871484894740709
/data/fan/UsersInTokyoProcessed/20121203_interp.pk
2012 12 3 2.615577859968097 0.6722207319338629 91.08941339437014 0.7835743108976313
/data/fan/UsersInTokyoProcessed/20121204_interp.pk
2012 12 4 2.6628923766356203 0.6653690200487313 93.4432385040965 0.7755608126492729
/data/fan/UsersInTokyoProcessed/20121205_interp.pk
2012 12 5 2.7366292154261704 0.6538825341690018 96.8276617159023 0.7649852806944369
/data/fan/UsersInTokyoProcessed/20121206_interp.pk
2012 12 6 2.7248155126891027 0.6565433687153175 96.83862872245429 0.766690738887324
/data/fan/UsersInTokyoProcessed/20121207_interp.pk
2012 12 7 2.8181888195409006 0.6425508333428112 99.36618282220981 0.7524105583346206
/data/fan/UsersInTokyoProcessed/20121208_interp.pk
2012

2013 1 30 2.8459449464556856 0.6401127955913511 107.82967866501173 0.7506237384185254
/data/fan/UsersInTokyoProcessed/20130131_interp.pk
2013 1 31 2.825739041171969 0.6445784540618507 106.36195717636618 0.7535517306960822
/data/fan/UsersInTokyoProcessed/20130201_interp.pk
2013 2 1 2.92033288853994 0.6302514036697643 112.04877095146115 0.7394868839180011
/data/fan/UsersInTokyoProcessed/20130202_interp.pk
2013 2 2 3.357622585319869 0.5533762462794275 136.7759554745914 0.683400275083037
/data/fan/UsersInTokyoProcessed/20130203_interp.pk
2013 2 3 3.3319549202726324 0.5487062571025728 132.4476770966999 0.6905753380712664
/data/fan/UsersInTokyoProcessed/20130204_interp.pk
2013 2 4 2.8502317049693637 0.6413013445640124 109.22967456451907 0.7509328236400199
/data/fan/UsersInTokyoProcessed/20130205_interp.pk
2013 2 5 2.876732596022573 0.6377127417672873 111.05715159693258 0.7462624480218418
/data/fan/UsersInTokyoProcessed/20130206_interp.pk
2013 2 6 2.7977926639584108 0.6449042106241087 99.4721

In [16]:
with open('./results_tokyo/pmm_metrics.pk', 'wb') as f:
    pk.dump(metrics_dict, f)

In [18]:
metrics_dict

{(2012, 12, 1): (3.3041999275062275,
  0.5584707496133254,
  129.328559622196,
  0.6897225501770956),
 (2012, 12, 2): (3.34368893869176,
  0.546077038407195,
  128.89247451166057,
  0.6871484894740709),
 (2012, 12, 3): (2.615577859968097,
  0.6722207319338629,
  91.08941339437014,
  0.7835743108976313),
 (2012, 12, 4): (2.6628923766356203,
  0.6653690200487313,
  93.4432385040965,
  0.7755608126492729),
 (2012, 12, 5): (2.7366292154261704,
  0.6538825341690018,
  96.8276617159023,
  0.7649852806944369),
 (2012, 12, 6): (2.7248155126891027,
  0.6565433687153175,
  96.83862872245429,
  0.766690738887324),
 (2012, 12, 7): (2.8181888195409006,
  0.6425508333428112,
  99.36618282220981,
  0.7524105583346206),
 (2012, 12, 8): (3.419343489461669,
  0.5428568243139492,
  138.06513724732133,
  0.6746019965757207),
 (2012, 12, 9): (3.3771051727016905,
  0.5418300347365438,
  131.12818902478747,
  0.68336841633823),
 (2012, 12, 10): (2.7806070541746086,
  0.6488689166963285,
  100.08675060644565,